## Select the top 200 Most Occurring Unknown genes and the 200 Least Occurring Unknown genes

We are simply looking at the count of occurrences and not the level of expression

In [4]:
# If you do not have the data you can get it from here
# Put the data in the output_dir so that it does not get saved to git hub
!wget -P ./input_data http://www.wormcat.com/static/download/whole_genome_v2_nov-11-2021.csv

--2023-03-31 13:44:44--  http://www.wormcat.com/static/download/whole_genome_v2_nov-11-2021.csv
Resolving www.wormcat.com (www.wormcat.com)... 3.85.241.50
Connecting to www.wormcat.com (www.wormcat.com)|3.85.241.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8471921 (8.1M) [text/csv]
Saving to: ‘./input_data/whole_genome_v2_nov-11-2021.csv’

whole_genome_v2_nov 100%[===================>]   8.08M  11.5MB/s    in 0.7s    

2023-03-31 13:44:44 (11.5 MB/s) - ‘./input_data/whole_genome_v2_nov-11-2021.csv’ saved [8471921/8471921]



In [5]:
# Load worm cat data and select the Unassigned

import anndata as ad
import pandas as pd
from scipy.sparse import csr_matrix

# Note if you dont find CSV file look in 'backup-runs' directory
worm_cat_df = pd.read_csv('./input_data/whole_genome_v2_nov-11-2021.csv') 
unassigned_df = worm_cat_df[worm_cat_df['Category 1']=='Unassigned']
unassigned_df.reset_index(drop=True, inplace=True)
print(f"We have {len(unassigned_df):,} Unassigned genes in Category 1")


We have 6,343 Unassigned genes in Category 1


In [6]:
# Load the h5ad file and set common variables
ad_worm_aging = ad.read("./input_data/ad_worm_aging.h5ad")

x_df = pd.DataFrame(data=csr_matrix.todense(ad_worm_aging.X))

obs_df = ad_worm_aging.obs
obs_df.reset_index(drop=True, inplace=True)

var_df = ad_worm_aging.var
var_df.reset_index(drop=True, inplace=True)


In [7]:
# Get a list of unique cell types
cell_types = obs_df['annotate_name'].unique()
print(f"We have {len(cell_types)} unique Cell types")

We have 211 unique Cell types


In [9]:
# find index in x_df for Wormbase ID (column position)
# each column in the x_df is a gene. we are mapping the column position so we can index to the associated gene
import warnings
#warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

def find_index(row):
    ret_val = var_df.index[var_df['gene_ids'] == row['Wormbase ID']].tolist()
    if ret_val == []:
        ret_val = None
    else:
        ret_val = ret_val[0]
    return ret_val

unassigned_df['gene_index'] = unassigned_df.apply(lambda row: find_index(row), axis=1)
print(f"We have {unassigned_df['gene_index'].isna().sum()} Unassigned genes that are not represented in the dataset")
print(f"That is %{round(unassigned_df['gene_index'].isna().sum()/len(unassigned_df)*100,2)} of Unassigned genes")

# unassigned_in_ds_df is the unassigned that are also in the dataset
unassigned_in_ds_df = unassigned_df[unassigned_df['gene_index'].notna()]
print(f"After removing the not represented rows we have {len(unassigned_in_ds_df['gene_index']):,} Unassigned genes")

We have 958 Unassigned genes that are not represented in the dataset
That is %15.1 of Unassigned genes
After removing the not represented rows we have 5,385 Unassigned genes


/tmp/ipykernel_917125/3168770814.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unassigned_df['gene_index'] = unassigned_df.apply(lambda row: find_index(row), axis=1)


In [10]:
# For each unknown gene count the number of non-zero entries for all cells/observations
# the result is a dictionary with gene index as key and cell_counts as value

# NOTE: This will take a few minutes to run!!
import time

results_dict={}
start = time.time()
for index in unassigned_in_ds_df['gene_index']:
    cell_count = x_df[x_df[index]>0].count()[index]
    results_dict[index]=cell_count

end = time.time()
m, s = divmod(round(end - start), 60)
h, m = divmod(m, 60)
print(f"Time for call hours {h:d} minutes {m:02d} seconds {s:02d} ")
results_dict_sorted = {k: v for k, v in sorted(results_dict.items(), key=lambda item: item[1])}


Time for call hours 0 minutes 03 seconds 28 


In [12]:
# Select the least occuring 200 genes and the most occuring 200 genes

results_lst_sorted = list(results_dict_sorted.keys())
first_200 = results_lst_sorted[0:200]
last_200 = results_lst_sorted[-200:]

def find_wormbase_id(index):
    ret_val = var_df.loc[index,['gene_ids']]
    return ret_val[0]

first_200_lst = []
for index in first_200:
    first_200_lst.append(find_wormbase_id(index))
first_200_df = pd.DataFrame (first_200_lst, columns = ['wormbase_id'])
first_200_df.to_csv("./output_data/least_occurring_unknown.csv",index=False)
    
last_200_lst = []
for index in last_200:
    last_200_lst.append(find_wormbase_id(index))
last_200_df = pd.DataFrame (last_200_lst, columns = ['wormbase_id'])
last_200_df.to_csv("./output_data/most_occurring_unknown.csv",index=False)

## Select all Unassigned: membrane spanning domain genes from wormcat

In [13]:
import random

worm_cat_df = pd.read_csv('./input_data/whole_genome_v2_nov-11-2021.csv') 

unassigned_msd_df = worm_cat_df[worm_cat_df['Category 3']=='Unassigned: membrane spanning domain']
unassigned_msd_series = unassigned_msd_df['Wormbase ID']
unassigned_msd_df.reset_index(drop=True, inplace=True)
unassigned_msd_ids_df = unassigned_msd_series.to_frame()
unassigned_msd_ids_df.rename(columns={'Wormbase ID': 'wormbase_id'}, inplace=True)
unassigned_msd_ids_df.to_csv('membrane_unassigned.csv',index=False)
#print(unassigned_msd_ids_df)

## Select 200 Random Unssigned Genes from Wormcat Category 1

In [14]:
# The genes below are Worm Cat Category 1 Unassigned
import random

worm_cat_df = pd.read_csv('./input_data/whole_genome_v2_nov-11-2021.csv') 

unassigned_df = worm_cat_df[worm_cat_df['Category 1']=='Unassigned']
unassigned_df.reset_index(drop=True, inplace=True)


def select_random(unassigned_df, max_genes=200):
    random_unassigned = {}
    while len(random_unassigned) < max_genes:
        index = random.randint(0, unassigned_df.shape[0])
        if index in random_unassigned.keys():
            random_unassigned[index] +=1
        else:
            random_unassigned[index] =1

    random_unassigned_df = pd.DataFrame() 
    random_unassigned_lst = []
    #wormbase_id
    for index in random_unassigned.keys():
        random_unassigned_lst.append(unassigned_df.loc[index,['Wormbase ID']][0])
    
    random_unassigned_df['wormbase_id'] = random_unassigned_lst
    return random_unassigned_df
    
random_unassigned_df = select_random(unassigned_df)

random_unassigned_df.to_csv('./output_data/random_unassigned.csv',index=False)
print(random_unassigned_df)

        wormbase_id
0    WBGene00014253
1    WBGene00004116
2    WBGene00009868
3    WBGene00012836
4    WBGene00195046
..              ...
195  WBGene00303071
196  WBGene00011624
197  WBGene00022746
198  WBGene00008039
199  WBGene00010917

[200 rows x 1 columns]


## Hypothesis Cell Types

* Discover unknown genes that are isolated to a specific cell type

* Discover unknown genes that are pervasive across cell types

* Identify cell types with no uknown genes


In [15]:
#unknown_df = unassigned_df.copy(deep=True)
cell_types = obs_df['annotate_name'].unique()

In [16]:
# Create a dictionary with all cell types and a list of observations that the cell type appears in

cell_type_dict = {}
for cell_type in cell_types:
    cells = obs_df[obs_df['annotate_name']==cell_type]
    cell_type_dict[cell_type] = cells.index.values.tolist()
    
#cell_type_dict

In [17]:
import numpy as np


# Take a single observation and count the unknown genes that are present
def unknown_gene_count(cell_attribs, unknown_genes_array, unassigned_in_ds_df):
    cell_attribs_lst = cell_attribs.tolist()
    
    index=0
    for gene_pos in unassigned_in_ds_df['gene_index']:
        if cell_attribs_lst[int(gene_pos)] > 0: #If gene is expressed in the cell at any level increment count
            unknown_genes_array[index] += 1
        index +=1
    return unknown_genes_array
        

# Used for testing    
# cell_attribs = x_df.loc[0]
# print(len(cell_attribs))
# unknown_genes_array = np.zeros(len(unassigned_df['gene_index']), dtype = int)
# ret_val = unknown_gene_count(cell_attribs,unknown_genes_array)
# print(len(ret_val))
# print(ret_val.sum())

In [18]:
import time
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

start = time.time()
# Create a copy of unassigned_in_ds_df and prepare to append cell type data
unassigned_mapped_to_cell_type_df = unassigned_in_ds_df.copy(deep=True)
unassigned_mapped_to_cell_type_df.drop(['Category 1','Category 2','Category 3','Automated Description'], inplace=True, axis=1)

for key in cell_type_dict.keys():
    unknown_genes_array = np.zeros(len(unassigned_in_ds_df['gene_index']), dtype = int)
    for observation in cell_type_dict[key]:
        cell_attribs = x_df.loc[observation]
        unknown_genes_array = unknown_gene_count(cell_attribs, unknown_genes_array, unassigned_in_ds_df)
    unassigned_mapped_to_cell_type_df[key]=unknown_genes_array

end = time.time()
m, s = divmod(round(end - start), 60)
h, m = divmod(m, 60)
print(f"Time for call hours {h:d} minutes {m:02d} seconds {s:02d} ")    
unassigned_mapped_to_cell_type_df.to_csv("./output_data/unassigned_mapped_to_cell_type_df.csv",index=False)

Time for call hours 0 minutes 00 seconds 58 


In [19]:
# Identify cell types with no uknown genes

# All Cell Types have some level of interaction with Unknown Genes
# '3_3:germline, sperm' has the lowest level with 320 counts this makes sense since there are only 4 observation that make up this cell type [2003, 15156, 17008, 40736]
# 0_0:seam cell has the highest with 321,367 this may also make sense since 2,202 observations contrubute to this cell type

cell_types_gene_count_dict = {}
for key in cell_type_dict.keys():
    count = unassigned_mapped_to_cell_type_df[key].sum()
    cell_types_gene_count_dict[key]= count

cell_types_gene_count_dict_sorted = {k: v for k, v in sorted(cell_types_gene_count_dict.items(), key=lambda item: item[1])}
for key in cell_types_gene_count_dict_sorted.keys():
    print(f"{key:<60} {cell_types_gene_count_dict_sorted[key]:>5,}")
    


3_3:germline, sperm                                            320
82_1:excretory system, hypodermis, arcade, sheath and socket   427
13_1:uterine epithelial cell                                   478
145_0:VC motor neurons 4-5                                   1,553
146_0:uterine-vulval cell                                    1,851
30_5:OLQ/IL sheath?                                          2,113
38_3:NA muscle                                               2,173
3_4:intestine                                                2,888
30_4:pharyngeal-intestinal valve                             3,286
14_2:AS11/VA/PDB motor neurons                               3,460
82_0:excretory pore, hypodermis, seam, vulF                  3,914
3_0:germline                                                 4,101
36_2:neurons                                                 4,110
144_0:RIS inter neurons                                      4,270
3_1:intestine                                                4

In [20]:
# Discover unknown genes that are isolated to a specific cell type

# Start by creating a temp dataframe with just the Wormbased IDs of the unknowns
temp_df = unassigned_mapped_to_cell_type_df['Wormbase ID'].copy().to_frame()
print(len(temp_df))

# A simple function to sum up the colums related to cell types
def sum_category_counts(row):
     return row[3:].sum()
    
# Create a new column named counts with total for all indivudual cell types
temp_series =  unassigned_mapped_to_cell_type_df.apply(lambda row: sum_category_counts(row), axis=1)
temp_df['counts'] = temp_series

# Sort the data by 'counts' and write to a csv file
temp_df = temp_df.sort_values('counts')
temp_df.to_csv('./output_data/isolated2cellType.csv',index=False)
temp_df

5385


,Wormbase ID,counts
2318,WBGene00235367,1
1696,WBGene00010129,1
391,WBGene00007673,1
1690,WBGene00010103,1
380,WBGene00044668,1
...,...,...
842,WBGene00009584,22382
4527,WBGene00004131,23356
3110,WBGene00012140,26606
1579,WBGene00009888,28240


In [21]:
# Given a row with a wormbase ID look up the same 'Wormbase ID' in the unassigned_mapped_to_cell_type_df dataframe
# This dataframe has the counts of the number of times that the 'Wormbase ID' was expressed in any observation for this cell type
# It does not include the level of expression only if the gene was on or not

cell_type_list = list(cell_type_dict.keys())

#This is a complete hack peace of sh#$%@&t
def cell_types(row_a):
    result = unassigned_mapped_to_cell_type_df.loc[unassigned_mapped_to_cell_type_df['Wormbase ID']==row_a['Wormbase ID']]
    row = result.iloc[0]
    #print(row_a['Wormbase ID'])
    cell_type_lst = []
    pos=0
    for attrib in row:
        pos +=1
        if pos > 3 and attrib >0:
            cell_type_lst.append(cell_type_list[pos-4])
    
    print(cell_type_lst)
    return cell_type_lst


   

In [23]:
# Here we get down to the details of unknown genes that are isolated to a specific cell type
import warnings
#warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

temp_df_20 = temp_df.loc[temp_df['counts']<3]
temp_df_20['cell_types'] =  temp_df_20.apply(lambda row: cell_types(row), axis=1)
temp_df_20.to_csv('./output_data/isolated2cellType.csv',index=False)

['20_2:PLM neurons']
['3_4:intestine']
['3_5:intestine']
['34_0:NA glia']
['64_0:hypodermis']
['3_4:intestine']
['57_1:germline']
['12_0:coelomocyte']
['4_0:BW muscles']
['90_0:AUA neurons']
['41_0:pharyngeal gland']
['36_2:neurons']
['135_0:I6 inter neurons']
['38_3:NA muscle']
['82_0:excretory pore, hypodermis, seam, vulF']
['131_0:AVG inter neurons ']
['26_0:pharyngeal muscle 7']
['7_4:inestinal, anal and uterine muscles']
['3_5:intestine']
['3_4:intestine']
['13_1:uterine epithelial cell']
['3_5:intestine']
['3_4:intestine']
['41_0:pharyngeal gland']
['3_4:intestine']
['1_1:BW muscles']
['41_0:pharyngeal gland']
['36_2:neurons']
['34_0:NA glia']
['3_4:intestine']
['140_0:ALA neurons']
['26_0:pharyngeal muscle 7']
['84_0:ADF amphid neurons']
['31_0:DTC distal tip', '30_2:hypodermis?']
['2_0:posterior hypodermis', '11_1:uterus vulva interface']
['63_0:ASK amphid neurons', '3_5:intestine']
['1_1:BW muscles', '14_2:AS11/VA/PDB motor neurons']
['93_0:ADA inter neurons', '35_0:AFD amphid

/tmp/ipykernel_917125/1509177464.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df_20['cell_types'] =  temp_df_20.apply(lambda row: cell_types(row), axis=1)


## Select all Category 1 'Muscle function' genes from wormcat

In [25]:
# Load worm cat data and select the unknowns

import anndata as ad
import pandas as pd
from scipy.sparse import csr_matrix

worm_cat_df = pd.read_csv('./input_data/whole_genome_v2_nov-11-2021.csv') 
muscle_function_df = worm_cat_df[worm_cat_df['Category 1']=='Muscle function']
muscle_function_df.drop(['Sequence ID','Category 1','Category 2','Category 3','Automated Description'], inplace=True, axis=1)
muscle_function_df.rename(columns={'Wormbase ID': 'wormbase_id'}, inplace=True)
muscle_function_df.reset_index(drop=True, inplace=True)
muscle_function_df.to_csv('./output_data/muscle_function.csv',index=False)
print(f"Muscle function = {len(muscle_function_df):,}")
print(muscle_function_df)

Muscle function = 62
       wormbase_id
0   WBGene00010685
1   WBGene00006495
2   WBGene00015061
3   WBGene00006441
4   WBGene00007044
..             ...
57  WBGene00006819
58  WBGene00006820
59  WBGene00006825
60  WBGene00006827
61  WBGene00021558

[62 rows x 1 columns]


/tmp/ipykernel_917125/3091621786.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muscle_function_df.drop(['Sequence ID','Category 1','Category 2','Category 3','Automated Description'], inplace=True, axis=1)
/tmp/ipykernel_917125/3091621786.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muscle_function_df.rename(columns={'Wormbase ID': 'wormbase_id'}, inplace=True)
